# Hey README, tell me your secrets 

## Plan

Goal: Scrape READMEs in github to determine their respective programing language

To Do: 
- Find readmes to scrape, at least 100 
- Pull content

Findings: 

In [2]:
# data wrangling
import pandas as pd
import numpy as np
import json

#visuals
import matplotlib.pyplot as plt
import seaborn as sns

# modules
import acquire
import prepare
#import explore
import model

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Acquire

A list of 100 pages to scrape are generated using the `acquire.loop_through_urls` function. They are saved to a csv and that csv file is being read for further manipulation. 

In [ ]:
# to_scrape = acquire.loop_through_urls()
# to_scrape = pd.DataFrame(to_scrape)
# to_scrape.to_csv("URL_list_100_final.csv")

In [ ]:
# read in list of pages generated from function 
pd.read_csv("URL_list_100.csv", header=0, names=["page"])

The dataframe of 100 pages that was just created is looped through, and on each page, it's respective repository, language, and readme is extracted as a dictionary, using the `acquire.make_corpus` function. 

The `acquire.get_corpus` fuction checks to see if the data file is in the cache. If it is, it reads it in as a dataframe, if file is not in cache, the above mentioned function is run to generate it.

In [ ]:
df_init = acquire.get_corpus()

In [ ]:
df_init

## Prepare

In [ ]:
# make a copy for prepping and exploring
df = df_init.copy()

In [ ]:
df.language.value_counts(dropna=False)

Using the `prepare.cut_singles` function, remove the languages that only have a single readme represented.

In [ ]:
df = prepare.cut_singles(df)

In [ ]:
df.language.value_counts()

In [ ]:
# prepare.prep_readme(df)

In [ ]:
df["readme"] = df.readme.apply(prepare.better_clean).apply(prepare.remove_stopwords).apply(prepare.lemmatize)

In [ ]:
df

## Explore

In [ ]:
# what is the breakdown of programming languages 
explore.count_percent(df)

In [ ]:
explore.readme_length(df)

In [ ]:
explore.readme_length(df).plot.bar()
plt.show()

### Model

Model will use the entire 100 rows of corpus. It is then stripped of unnecssary characters and brokend down into 

components with lemmatization.

In [ ]:
# df = acquire.get_corpus('data_final.json')
# df['prepared'] = df.readme.apply(prepare.basic_clean).apply(prepare.remove_stopwords).apply(prepare.lemmatize)
# df = prepare.cut_singles(df)

In [3]:
df1 = acquire.get_corpus('data_final.json')
df2 = pd.read_json('data_second.json')
frames = [df1,df2]
df = pd.concat(frames)

In [27]:
df['prepared'] = df.readme.apply(prepare.basic_clean).apply(prepare.remove_numbers).apply(prepare.remove_stopwords).apply(prepare.lemmatize)
df = prepare.cut_singles(df)

The data is broken into training and testing sets. In this instance it will be a 70/30 split.

In [28]:
X_train, X_test, train, test = model.make_model_components(df.prepared, df.language, .25)

In [29]:
classifier = model.make_tree_model(X_train, train.actual)

In [30]:
training_predictions = classifier.predict(X_train)
train['predictions'] = training_predictions
test_predictions = classifier.predict(X_test)
test['predictions'] = test_predictions

#### Accuracy scores for the model on the Training set

In [31]:
model.score_your_model(train.actual, train.predictions)

Accuracy: 76.61%
---
Confusion Matrix
actual       C  C++  CSS  Go  Java  JavaScript  None  Python  TypeScript  Vue
predictions                                                                  
C++          0    2    0   0     0           0     0       1           0    0
CSS          0    0    2   0     0           0     0       0           0    0
Go           3    4    1   7     1           2     4       3           3    0
Java         0    0    0   0    10           0     0       0           0    0
JavaScript   0    0    1   0     0          41     0       0           0    0
None         0    0    0   0     0           0    17       0           0    0
Python       0    3    0   0     0           1     0      12           0    0
TypeScript   0    0    0   0     0           0     0       0           4    2
---
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
         C++       0.67      0.22      0.33         9
         CSS   

#### Accuracy scores for the model on the Test set

In [32]:
model.score_your_model(test.actual, test.predictions)

Accuracy: 45.24%
---
Confusion Matrix
actual       C  C++  CSS  Go  Java  JavaScript  None  Python  TypeScript  Vue
predictions                                                                  
C++          0    0    0   1     0           0     0       0           0    0
Go           1    1    0   1     1           1     3       1           1    0
Java         0    0    0   0     3           0     0       0           0    0
JavaScript   0    1    1   0     0          10     2       0           1    0
None         0    0    0   0     0           3     3       2           0    0
Python       0    1    0   0     0           1     0       2           0    0
TypeScript   0    0    0   0     0           0     0       0           0    1
---
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         1
         C++       0.00      0.00      0.00         3
         CSS       0.00      0.00      0.00         1
          Go       0.10      0.50      